In [1]:
import tensorflow as tf
import os
import sys
print (sys.path)
sys.path.append('/data_preprocessing')
from tensorflow import keras as ke
import numpy as np
import random as rd
from list_to_onehot import *
import multiprocessing as mp

vocab_size = 10000
dim = 200
no_ofppl = 20
input_sentsize = 30
input_size_bot = dim*input_sentsize + no_ofppl

['c:\\Users\\rohan\\Desktop\\Parth\\humantextpatterncloner\\data_preprocessing', 'c:\\Users\\rohan\\anaconda3\\envs\\tf\\python39.zip', 'c:\\Users\\rohan\\anaconda3\\envs\\tf\\DLLs', 'c:\\Users\\rohan\\anaconda3\\envs\\tf\\lib', 'c:\\Users\\rohan\\anaconda3\\envs\\tf', '', 'c:\\Users\\rohan\\anaconda3\\envs\\tf\\lib\\site-packages', 'c:\\Users\\rohan\\anaconda3\\envs\\tf\\lib\\site-packages\\win32', 'c:\\Users\\rohan\\anaconda3\\envs\\tf\\lib\\site-packages\\win32\\lib', 'c:\\Users\\rohan\\anaconda3\\envs\\tf\\lib\\site-packages\\Pythonwin']


In [2]:
#word2vec

def word2vec(vocab_size = vocab_size,h_size = dim):
    model = ke.Sequential()
    model.add(ke.layers.Dense(h_size, input_shape = (vocab_size,)))
    model.add(ke.layers.Dense(vocab_size,activation = ke.activations.softmax))
    return model

In [3]:
#model creation
word2vec_model = word2vec()
word2vec_model.compile(loss="categorical_crossentropy",optimizer='adam', metrics=['accuracy'])

In [4]:
#useful functions

def func(sub_batch,L,id):
    for n in sub_batch:
        x_file = open("arraydata/x"+n) 
        y_file = open("arraydata/y"+n)
        x_n = np.load(x_file)
        y_n = np.load(y_file)
        L[id].append((x_n,y_n))
        x_file.close()
        y_file.close()

def load_batch_array(batch:list,processes = 6):
    manager = mp.Manager()
    L = manager.list()
    # Ly = manager.list()
    for i in range(processes):
        L.append([])
        # Ly.append([])

    no_of_paths = len(batch)
    quot = no_of_paths//processes
    rem = no_of_paths%processes
    sub_batch_sizes = []
    for i in range(processes):
        if rem != 0:
            sub_batch_sizes.append(quot+1)
            rem -= 1
        else:
            sub_batch_sizes.append(quot)

    S = 0
    proc_list = []
    for j in range(processes):
        sub_batch = batch[S:S+sub_batch_sizes[j]]
        S += sub_batch_sizes[j]
        id = j
        p = mp.Process(target=func,args=(sub_batch,L,id))
        proc_list.append(p)
        p.start()

    for p in proc_list:
       p.join()
    x_batch_list = []
    y_batch_list = []
    for l in L:
        for (x,y) in l:
            x_batch_list.append(x)
            y_batch_list.append(y)
    x_batch_arr = np.array(x_batch_list)
    y_batch_arr = np.array(y_batch_list)
    return x_batch_arr,y_batch_arr

In [24]:
#training loop for when we save each array individually
import random
no_of_epochs = 1
batch_size = 50
data_total_size = 307895

L = [str(x) for x in range(data_total_size)]
L_shuffle = list(L)
for epoch in range(1,no_of_epochs+1):
    random.shuffle(L_shuffle)
    for i in range(data_total_size//batch_size):
        batch = L_shuffle[i*batch_size:(i+1)*batch_size]
        print("batch,",batch)
        # x_batch_list = []
        # y_batch_list = []
        # for n in batch: #parallelise this loop
        #     x_file = open("arraydata/x"+n) 
        #     y_file = open("arraydata/y"+n)
        #     x_n = np.load(x_file)
        #     y_n = np.load(y_file)
        #     x_batch_list.append(x_n)
        #     y_batch_list.append(y_n)
        #     x_file.close()
        #     y_file.close()
        # x_batch_array = np.array(x_batch_list)
        # y_batch_array = np.array(y_batch_list)
        x_batch_array, y_batch_array = load_batch_array(batch)
        word2vec_model.train_on_batch(x_batch_array,y_batch_array)
        

batch, 50


ValueError: Data cardinality is ambiguous:
  x sizes: 0
  y sizes: 6
Make sure all arrays contain the same number of samples.

In [ ]:
word2vec_layer = word2vec_model.layers[0]
word2vec_layer.trainable = False

def tokens_to_vecs(sent:list,layer):
    onehot_list = []
    for word in sent:
        onehot_list.append(convertInputToOneHotPercentages(word))
    onehot_array = np.array(onehot_list)
    vec_array = layer.predict(onehot_array)
    return vec_array



